<h1>Notebook to clean the Tigers Data Set</h1>
<p>data/DTIFanData_2-21-25.csv will be cleaned</p>

<h3>Dependencies</h3>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

<h3>Load the dataset</h3>

In [3]:
path = 'data/DTIFanData_2-21-25.csv'
# Load the data
data = pd.read_csv(path)
# Display the first few rows of the dataframe
print(data.head())

/tmp/ipykernel_1154/3605361889.py:3: DtypeWarning: Columns (4,6,7,165) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


   KeepFlag  GlobalKey  SeasonKey FanSinceDate  \
0         1         11       2025   2009-05-01   
1         1         12       2025   2009-02-25   
2         1         15       2025   2008-08-05   
3         1         17       2025   2007-06-24   
4         1         19       2025   2008-08-05   

                  FirstGameAttended  TotalGamesAttended  \
0         2023-08-05 Tampa Bay Rays                 4.0   
1  2024-06-25 Philadelphia Phillies                 1.0   
2     2013-06-27 Los Angeles Angels                 6.0   
3      2015-07-05 Toronto Blue Jays                 9.0   
4          2013-07-14 Texas Rangers                 1.0   

                    FirstGameBought                    LastGameBought  \
0         2023-08-05 Tampa Bay Rays  2024-06-26 Philadelphia Phillies   
1  2024-06-25 Philadelphia Phillies  2024-06-25 Philadelphia Phillies   
2       2021-06-08 Seattle Mariners       2021-06-08 Seattle Mariners   
3  2021-06-26 Houston Astros Game 2      2024-09-29 

In [4]:
# Function to check for type errors in columns
def check_type_errors(df):
    type_errors = {}
    for column in df.columns:
        try:
            df[column].astype(df[column].dtype)
        except ValueError as e:
            type_errors[column] = str(e)
    return type_errors

# Check for type errors in the data
type_errors = check_type_errors(data)
print("Columns with type errors:", type_errors)

Columns with type errors: {}


In [6]:
# Get the data types of all columns
column_types = data.dtypes
for column, dtype in column_types.items():
    print(f"{column}: {dtype}")

KeepFlag: int64
GlobalKey: int64
SeasonKey: int64
FanSinceDate: object
FirstGameAttended: object
TotalGamesAttended: float64
FirstGameBought: object
LastGameBought: object
TotalTicketsPurchased: float64
TotalLifetimeValue: float64
CurrentSeasonEmailActivities: float64
PreviousSeasonsEmailActivities: float64
STMFlagCurr: int64
TicketingFanType: object
EmailFanType: object
FullSeasonBuyer: float64
HalfSeasonBuyer: float64
QuarterSeasonBuyer: float64
MiniPlanBuyer: float64
IndividualGameBuyer: float64
City: object
State: object
PostalCd: object
Country: object
Gender: object
Education: object
Occupation: object
Age: float64
MaritalStatus: object
PresenceOfChildren: object
DwellingType: object
HouseholdIncome: object
NetWorth: object
PrimaryVehicleType: object
MSADescription: object
MailSuppresionFlg: float64
WorkingWomanFlg: float64
BankCardHolderFlg: float64
GasDepartmentRetailCardHolderFlg: float64
TravelEntertainmentCardHolderFlg: float64
CreditCardHolderUnknownTypeFlg: float64
Premium

In [9]:
state_counts = data['State'].value_counts()
state_counts = state_counts.reset_index()
state_counts.columns = ['State', 'Count']
print(state_counts.to_string())

                            State   Count
0                              MI  198554
1                              OH   11709
2                              ON    7507
3                        MICHIGAN    6377
4                              FL    6121
5                              NY    4368
6                              CA    3742
7                              IL    3582
8                              IN    3166
9                              TX    2537
10                        FLORIDA    1762
11                           OHIO    1703
12                             PA    1691
13                       ILLINOIS    1310
14                             MA    1262
15                             NC    1234
16                             VA    1229
17                             AZ    1188
18                             GA    1173
19                     CALIFORNIA    1160
20                        ONTARIO    1154
21                             TN    1036
22                             NJ 

In [15]:
# Mapping dictionary for state normalization
state_mapping = {
    'MI': 'MICHIGAN',
    'OH': 'OHIO',
    'IL': 'ILLINOIS',
    'IN': 'INDIANA',
    'WI': 'WISCONSIN',
    'MN': 'MINNESOTA',
    'IA': 'IOWA',
    'KY': 'KENTUCKY',
    'MO': 'MISSOURI',
    'PA': 'PENNSYLVANIA',
    'NY': 'NEW YORK',
    'NJ': 'NEW JERSEY',
    'CT': 'CONNECTICUT',
    'MD': 'MARYLAND',
    'VA': 'VIRGINIA',
    'NC': 'NORTH CAROLINA',
    'SC': 'SOUTH CAROLINA',
    'GA': 'GEORGIA',
    'FL': 'FLORIDA',
    'TX': 'TEXAS',
    'CA': 'CALIFORNIA',
    'WA': 'WASHINGTON',
    'OR': 'OREGON',
    'CO': 'COLORADO',
    'AZ': 'ARIZONA',
    'NV': 'NEVADA',
    'UT': 'UTAH',
    'ID': 'IDAHO',
    'MT': 'MONTANA',
    'WY': 'WYOMING',
    'ND': 'NORTH DAKOTA',
    'SD': 'SOUTH DAKOTA',
    'NE': 'NEBRASKA',
    'KS': 'KANSAS',
    'LA': 'LOUISIANA',
    'AR': 'ARKANSAS',
    'MS': 'MISSISSIPPI',
    'AL': 'ALABAMA',
    'TN': 'TENNESSEE',
    'WV': 'WEST VIRGINIA',
    'DE': 'DELAWARE',
    'VT': 'VERMONT',
    'NH': 'NEW HAMPSHIRE',
    'ME': 'MAINE',
    'HI': 'HAWAII',
    'AK': 'ALASKA',
    'NM': 'NEW MEXICO',
    'OK': 'OKLAHOMA',
    'MA': 'MASSACHUSETTS',
    'RI': 'RHODE ISLAND',
    # Non-state territories and regions
    'PR': 'PUERTO RICO',
    'DC': 'DISTRICT OF COLUMBIA',
    'AS': 'AMERICAN SAMOA',
    'GU': 'GUAM',
    'MP': 'NORTHERN MARIANA ISLANDS',
    'VI': 'VIRGIN ISLANDS',
    'FM': 'FEDERATED STATES OF MICRONESIA',
    'MH': 'MARSHALL ISLANDS',
    # International regions (if applicable)
    'OT': 'ONTARIO',  # Example for Ontario, Canada
    'ON': 'ONTARIO',  # Example for Ontario, Canada
    'Ontario': 'ONTARIO',  # Example for Ontario, Canada
    'NS': 'NOVA SCOTIA',  # Example for Nova Scotia, Canada
    'QC': 'QUEBEC',  # Example for Quebec, Canada
    'BC': 'BRITISH COLUMBIA',  # Example for British Columbia, Canada
    'AB': 'ALBERTA',  # Example for Alberta, Canada
    'SK': 'SASKATCHEWAN',  # Example for Saskatchewan, Canada
    'NL': 'NEWFOUNDLAND AND LABRADOR',  # Example for Newfoundland and Labrador, Canada
    'YT': 'YUKON',  # Example for Yukon, Canada
    'NT': 'NORTHWEST TERRITORIES',  # Example for Northwest Territories, Canada
    'NU': 'NUNAVUT'  # Example for Nunavut, Canada
    
}

# Function to normalize state values
def normalize_state(state):
    state = state.strip().upper()
    return state_mapping.get(state, state)

# Apply the normalization function to the 'State' column
data['State'] = data['State'].apply(lambda x: normalize_state(x) if pd.notnull(x) else x)

# Display the normalized state counts
normalized_state_counts = data['State'].value_counts().reset_index()
normalized_state_counts.columns = ['State', 'Count']
print(normalized_state_counts.to_string())

                            State   Count
0                        MICHIGAN  204931
1                            OHIO   13412
2                         ONTARIO    8661
3                         FLORIDA    7883
4                        NEW YORK    5013
5                      CALIFORNIA    4902
6                        ILLINOIS    4892
7                         INDIANA    3827
8                           TEXAS    3359
9                    PENNSYLVANIA    2152
10                       VIRGINIA    1778
11                 NORTH CAROLINA    1732
12                        ARIZONA    1656
13                        GEORGIA    1562
14                      TENNESSEE    1459
15                  MASSACHUSETTS    1430
16                     NEW JERSEY    1285
17                      WISCONSIN    1158
18                       COLORADO    1084
19                       MARYLAND     968
20                       MISSOURI     847
21                       KENTUCKY     841
22                 SOUTH CAROLINA 

In [16]:
# Get the counts of each state
state_counts = data['State'].value_counts()

# Function to update state values based on their occurrences
def update_state(state):
    if state_counts[state] <= 20 and state not in state_mapping.values():
        return 'OTHERS'
    return state

# Apply the update function to the 'State' column
data['State'] = data['State'].apply(lambda x: update_state(x) if pd.notnull(x) else x)

# Display the updated state counts
updated_state_counts = data['State'].value_counts().reset_index()
updated_state_counts.columns = ['State', 'Count']
print(updated_state_counts.to_string())

                        State   Count
0                    MICHIGAN  204931
1                        OHIO   13412
2                     ONTARIO    8661
3                     FLORIDA    7883
4                    NEW YORK    5013
5                  CALIFORNIA    4902
6                    ILLINOIS    4892
7                     INDIANA    3827
8                       TEXAS    3359
9                PENNSYLVANIA    2152
10                   VIRGINIA    1778
11             NORTH CAROLINA    1732
12                    ARIZONA    1656
13                    GEORGIA    1562
14                  TENNESSEE    1459
15              MASSACHUSETTS    1430
16                 NEW JERSEY    1285
17                  WISCONSIN    1158
18                   COLORADO    1084
19                   MARYLAND     968
20                     OTHERS     853
21                   MISSOURI     847
22                   KENTUCKY     841
23             SOUTH CAROLINA     821
24                 WASHINGTON     759
25          